## Pipeline Tutorial with HeteroSecureboost

### install

`Pipeline` is distributed along with [fate_client](https://pypi.org/project/fate-client/).

```bash
pip install fate_client
```

To use Pipeline, we need to first specify which `FATE Flow Service` to connect to. Once `fate_client` installed, one can find an cmd enterpoint name `pipeline`:

In [2]:
!pipeline --help

Usage: pipeline [OPTIONS] COMMAND [ARGS]...

Options:
  --help  Show this message and exit.

Commands:
  config  pipeline config tool
  init    - DESCRIPTION: Pipeline Config Command.


Assume we have a `FATE Flow Service` in 127.0.0.1:9380(defaults in standalone), then exec

In [3]:
!pipeline init --ip 127.0.0.1 --port 9380

Pipeline configuration succeeded.


### Hetero Secureboost Example

 Before start a modeling task, the data to be used should be uploaded. Please refer to this [guide](./pipeline_tutorial_upload.ipynb).

The `pipeline` package provides components to compose a `FATE pipeline`.

In [4]:
from pipeline.backend.pipeline import PipeLine
from pipeline.component import Reader, DataTransform, Intersection, HeteroSecureBoost, Evaluation
from pipeline.interface import Data

Make a `pipeline` instance:

    - initiator: 
        * role: guest
        * party: 9999
    - roles:
        * guest: 9999
        * host: 10000
    

In [5]:
pipeline = PipeLine() \
        .set_initiator(role='guest', party_id=9999) \
        .set_roles(guest=9999, host=10000)

Define a `Reader` to load data

In [6]:
reader_0 = Reader(name="reader_0")
# set guest parameter
reader_0.get_party_instance(role='guest', party_id=9999).component_param(
    table={"name": "breast_hetero_guest", "namespace": "experiment"})
# set host parameter
reader_0.get_party_instance(role='host', party_id=10000).component_param(
    table={"name": "breast_hetero_host", "namespace": "experiment"})

Add a `DataTransform` component to parse raw data into Data Instance

In [7]:
data_transform_0 = DataTransform(name="data_transform_0")
# set guest parameter
data_transform_0.get_party_instance(role='guest', party_id=9999).component_param(
    with_label=True)
data_transform_0.get_party_instance(role='host', party_id=[10000]).component_param(
    with_label=False)

Add a `Intersection` component to perform PSI for hetero-scenario

In [8]:
intersect_0 = Intersection(name="intersection_0")

Now, we define the `HeteroSecureBoost` component. The following parameters will be set for all parties involved.

In [9]:
hetero_secureboost_0 = HeteroSecureBoost(name="hetero_secureboost_0",
                                         num_trees=5,
                                         bin_num=16,
                                         task_type="classification",
                                         objective_param={"objective": "cross_entropy"},
                                         encrypt_param={"method": "iterativeAffine"},
                                         tree_param={"max_depth": 3})


To show the evaluation result, an "Evaluation" component is needed.

In [10]:
evaluation_0 = Evaluation(name="evaluation_0", eval_type="binary")

Add components to pipeline, in order of execution:

    - data_transform_0 comsume reader_0's output data
    - intersection_0 comsume data_transform_0's output data
    - hetero_secureboost_0 consume intersection_0's output data
    - evaluation_0 consume hetero_secureboost_0's prediciton result on training data

In [ ]:
pipeline.add_component(reader_0)
pipeline.add_component(data_transform_0, data=Data(data=reader_0.output.data))
pipeline.add_component(intersect_0, data=Data(data=data_transform_0.output.data))
pipeline.add_component(hetero_secureboost_0, data=Data(train_data=intersect_0.output.data))
pipeline.add_component(evaluation_0, data=Data(data=hetero_secureboost_0.output.data))


Then compile our pipeline to make it ready for submission.

In [ ]:
pipeline.compile()

Now, submit(fit) our pipeline:

In [13]:
pipeline.fit()

2021-11-15 11:27:50.488 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:123 - Job id is 202111151127428590790

2021-11-15 11:27:50.496 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:144 - Job is still waiting, time elapse: 0:00:00
2021-11-15 11:27:51.007 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:144 - Job is still waiting, time elapse: 0:00:00

2021-11-15 11:27:52.556 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component reader_0, time elapse: 0:00:02
2021-11-15 11:27:57.394 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component reader_0, time elapse: 0:00:06

2021-11-15 11:28:00.018 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component data_transform_0, time elapse: 0:00:09
2021-11-15 11:28:05.736 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component data_transfor

Once training is done, trained model may be used for prediction. Optionally, save the trained pipeline for future use.

In [ ]:
pipeline.dump("pipeline_saved.pkl")

First, deploy needed components from train pipeline

In [ ]:
pipeline = PipeLine.load_model_from_file('pipeline_saved.pkl')
pipeline.deploy_component([data_transform_0, intersect_0, hetero_secureboost_0])

Define new `Reader` components for reading prediction data

In [16]:
reader_1 = Reader(name="reader_1")
reader_1.get_party_instance(role="guest", party_id=9999).component_param(table={"name": "breast_hetero_guest", "namespace": "experiment"})
reader_1.get_party_instance(role="host", party_id=10000).component_param(table={"name": "breast_hetero_host", "namespace": "experiment"})

Optionally, define new `Evaluation` component.

In [17]:
evaluation_0 = Evaluation(name="evaluation_0", eval_type="binary")

Add components to predict pipeline in order of execution:

In [ ]:
predict_pipeline = PipeLine()
predict_pipeline.add_component(reader_1)\
                .add_component(pipeline, 
                data=Data(predict_input={pipeline.data_transform_0.input.data: reader_1.output.data}))\
                .add_component(evaluation_0, data=Data(data=pipeline.hetero_secureboost_0.output.data))

Then, run prediction job

In [19]:
predict_pipeline.predict()

2021-11-15 11:30:42.336 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:123 - Job id is 202111151130376077970

2021-11-15 11:30:42.393 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:144 - Job is still waiting, time elapse: 0:00:00
2021-11-15 11:30:49.101 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:144 - Job is still waiting, time elapse: 0:00:06
2021-11-15 11:30:49.613 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:144 - Job is still waiting, time elapse: 0:00:07

2021-11-15 11:30:50.665 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component reader_1, time elapse: 0:00:08
2021-11-15 11:30:55.006 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component reader_1, time elapse: 0:00:12
2021-11-15 11:30:55.529 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component reader_1, time elapse: 0:00:

For more demo on using pipeline to submit jobs, please refer to [pipeline demos](https://github.com/FederatedAI/FATE/tree/master/examples/pipeline/demo)